<a href="https://colab.research.google.com/github/yjkno1/AI_TEAM/blob/main/iForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import IsolationForest

from google.colab import drive
drive.mount('/gdrive', force_remount=True)



Mounted at /gdrive


In [37]:
df = pd.read_csv("/gdrive/My Drive/Colab Notebooks/file/10m_sample.csv")

df.describe()


,status,value
count,100000.000000,100000.000000
mean,0.023040,460.771877
std,5.151875,132.309891
min,0.000000,0.000000
25%,0.000000,300.000000
50%,0.000000,550.000000
75%,0.000000,557.000000
max,1152.000000,577.000000


In [31]:
# 랜덤 스테이트 수치 설정하기
state_value = np.random.RandomState(20)  
  
# IsolationForest 모델 만들기
isol = IsolationForest(max_samples=100, random_state=state_value) 

#x_train_df = df.filter(df.tag == "SRL_1.1YH70.ROUGHING_MILL_STAND1_ACTUAL_SPEED_CONTROL_VOLTAGE1").select("value").toPandas().values
# 100만건 == 47초
x_train_df = np.array(df.value)

x_train = x_train_df.reshape(-1,1)
print(x_train)


# x_train_df

# array([[32767.],
#       [32767.],
#       [32767.],
#       [32767.],
#       [32767.],
#       [32767.],
#       [32767.],
#       [32767.],
#       [32767.],


isol.fit(x_train)

# 만들어 둔 데이터셋에 알고리즘 피팅시키기
y_pred_train = isol.predict(x_train)

X = [[-1.1], [0.3], [0.5], [100]]
y_pred_train = isol.predict(X)

print(y_pred_train)
#y_pred_tran = ison.predict(df)

[[561. ]
 [557. ]
 [555. ]
 ...
 [217.9]
 [317.6]
 [265.3]]
[-1 -1 -1 -1]
